<a href="https://colab.research.google.com/github/Hou-jing/-repository-experiment/blob/main/bert_model_fune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import tensorflow as tf
train_sentence=['The system as described above has its greatest application in an arrayed <e1>configuration</e1> of antenna <e2>elements</e2>.', 'The <e1>child</e1> was carefully wrapped and bound into the <e2>cradle</e2> by means of a cord.']

In [64]:
train_relation=['Component-Whole', 'Other']

In [ ]:
from tensorflow.keras.preprocessing import text
#facts, accu_label, article_label, imprison_label=load_data()
somestr = train_sentence
tok=text.Tokenizer() #初始化标注器
token=tok.fit_on_texts(somestr) #学习出文本的字典
word_index = tok.word_index#查看对应的单词和数字的映射关系dict
print(word_index)
word_docs=tok.word_docs
print(word_docs)#document_count, word_counts, word_docs, index_docs, index_word, word_index

sequences = tok.texts_to_sequences(somestr) #通过texts_to_sequences 这个dict可以将每个string的每个词转成数字
print(sequences)

In [ ]:
!pip install transformers
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


In [42]:
def parse_sentence(sentence):
  bert_tks=[]

  for sentence in sentence:
    sentence=sentence.replace('<e1>', '<e1>').replace('</e1>', '<e1>').replace('<e2>', '<e1>').replace('</e2>', '<e1>')
    trunks=sentence.split('<e1>')
    bert_tokens=[]
    bert_tokens.append('[CLS]')
    if len(trunks)!=5:
      print('error')
    else:
      for i,trunk in enumerate(trunks):
        tks=tokenizer.tokenize(trunk)
        # tks=tok.fit_on_texts(trunk)
        bert_tokens.extend(tks)
        if i==0 or i==1:
          bert_tokens.append('[EN1]')
        elif i == 2 or i ==3:
                bert_tokens.append('[EN2]')

    bert_tokens.append("[SEP]")

    bert_tks.append(bert_tokens)
  return bert_tks
bert_token=parse_sentence(train_sentence)
print(bert_token)

[['[CLS]', 'the', 'system', 'as', 'described', 'above', 'has', 'its', 'greatest', 'application', 'in', 'an', 'array', '##ed', '[EN1]', 'configuration', '[EN1]', 'of', 'antenna', '[EN2]', 'elements', '[EN2]', '.', '[SEP]'], ['[CLS]', 'the', '[EN1]', 'child', '[EN1]', 'was', 'carefully', 'wrapped', 'and', 'bound', 'into', 'the', '[EN2]', 'cradle', '[EN2]', 'by', 'means', 'of', 'a', 'cord', '.', '[SEP]']]


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
seq=[tokenizer.convert_tokens_to_ids(toke) for toke in bert_token]
print(seq)

In [47]:
train_ids=pad_sequences(sequences=seq,maxlen=128,dtype='long',padding='post')
print(train_ids[0])

[  101  1996  2291  2004  2649  2682  2038  2049  4602  4646  1999  2019
  9140  2098   100  9563   100  1997 13438   100  3787   100  1012   102
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


In [63]:
import numpy as np
def build_entitymask(bert_tok):
  mask=[[i for i,x in enumerate(msk) if x=='[EN1]' or x=='[EN2]'] for msk in bert_tok]
  print(mask)
  '''mask=[]
  for mak in bert_tok:
    for x in range(len(mak)):
      if mak[x]=='[EN1]' of msk[x]=='[EN2]':
        mask.append(x)
        print(mask)'''
  e1_masks=[]
  e2_masks=[]
  for i,mark in enumerate(mask):
    e1_mask=np.zeros((128,))
    e2_mask=np.zeros((128,))
    print(mask[i][0]+1)
    print(mask[i][2]+1)
    e1_mask[mask[i][0]+1]=1
    e2_mask[mask[i][2]+1]=1
    e1_masks.append(e1_mask)
    e2_masks.append(e2_mask)
  return e1_masks,e2_masks
train_e1_mask,train_e2_mask=build_entitymask(bert_token)
print('训练数据第一个e1的mask{}'.format(train_e1_mask[0]))
print('测试数据第一个e2的mask{}'.format(train_e2_mask[0]))

[[14, 16, 19, 21], [2, 4, 12, 14]]
15
20
3
13
训练数据第一个e1的mask[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
测试数据第一个e2的mask[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [68]:
from sklearn.preprocessing.label import LabelEncoder
label=LabelEncoder()
label_rel=label.fit_transform(train_relation)#fit转为集合，transform转为编码
print(label_rel)
#查看编码对应的原始值
rel=label.inverse_transform([0,1])
print(rel)

[0 1]
['Component-Whole' 'Other']


In [71]:
#注意力mask
train_attention_mask=[[float(i>0) for i in ii] for ii in train_ids]
print (train_attention_mask[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [26]:
!pip install bert-tensorflow
!pip install tensorflow-hub
from bert import tokenization

                                                

     |████████████████████████████████| 64 kB 2.1 MB/s 


In [ ]:
import tensorflow_hub as hub
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
with tf.Graph().as_default():
  bert_module = hub.Module(BERT_MODEL_HUB)
  tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
  vocab_file, do_lower_case =tokenization_info["vocab_file"],tokenization_info["do_lower_case"]
Fulltoken=tokenization.FullTokenizer(vocab_file=vocab_file,do_lower_case=True)
token=Fulltoken.tokenize(train_sentence)

In [ ]:
import gensim
text='The system as described configuration'
token=gensim.utils.tokenize(text,lower=True)
print(token)

<generator object simple_tokenize at 0x7f909916f9d0>


In [ ]:
from tensorflow.keras.preprocessing import text
#facts, accu_label, article_label, imprison_label=load_data()
somestr = ['ha ha gua angry','howa ha gua excited naive']
tok=text.Tokenizer() #初始化标注器
tok.fit_on_texts(somestr) #学习出文本的字典
word_index = tok.word_index#查看对应的单词和数字的映射关系dict
print(word_index)
sequences = tok.texts_to_sequences(somestr) #通过texts_to_sequences 这个dict可以将每个string的每个词转成数字
print(sequences)


{'ha': 1, 'gua': 2, 'angry': 3, 'howa': 4, 'excited': 5, 'naive': 6}
[[1, 1, 2, 3], [4, 1, 2, 5, 6]]
